# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [1]:
# your code goes here
import pandas as pd

# Load the dataset (run only if you've downloaded it manually)
df = pd.read_csv("marketing_customer_analysis.csv")

# Step 1: Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.dropna(how='all', inplace=True)

# Step 2: Filter customers with total_claim_amount < 1000 and response == 'Yes'
filtered_df = df[(df['total_claim_amount'] < 1000) & (df['response'].str.lower() == 'yes')]

# Step 3: Average monthly_premium and customer_lifetime_value by policy_type and gender
yes_responders = df[df['response'].str.lower() == 'yes']
premium_clv_summary = yes_responders.groupby(['policy_type', 'gender'])[
    ['monthly_premium_auto', 'customer_lifetime_value', 'total_claim_amount']
].mean().reset_index()

# Step 4: Count customers per state and filter states with >500 customers
state_counts = df['state'].value_counts()
states_500_plus = state_counts[state_counts > 500]
filtered_states_df = df[df['state'].isin(states_500_plus.index)]

# Step 5: Max, Min, Median CLV by education and gender
clv_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()

# Display results
print("Filtered Customers (<$1000 & Responded 'Yes'):\n", filtered_df.head())
print("\nAverage Premium and CLV (Yes Responders):\n", premium_clv_summary)
print("\nStates with >500 Customers:\n", states_500_plus)
print("\nCLV Stats by Education and Gender:\n", clv_stats)



Filtered Customers (<$1000 & Responded 'Yes'):
     unnamed:_0 customer       state  customer_lifetime_value response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    coverage education effective_to_date employmentstatus gender  ...  \
3   Extended   College           1/11/11         Employed      M  ...   
8    Premium   College           1/19/11         Employed      M  ...   
15     Basic    Master           1/16/11         Employed      F  ...   
19  Extended   College           2/26/11         Employed      F  ...   
27   Premium  Bachelor           2/28/11         Employed      F  ...   

    number_of_open_complaints number_of_policies     policy_type  \
3     

In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv("marketing_customer_analysis.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Parse dates safely (adjust format if known)
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%m/%d/%Y', errors='coerce')
df['month'] = df['effective_to_date'].dt.month_name()

# Drop rows where state or month is missing
df = df.dropna(subset=['state', 'month'])

# Group and analyze
policies_by_state_month = df.groupby(['state', 'month']).size().reset_index(name='policy_count')
top_3_states = policies_by_state_month.groupby('state')['policy_count'].sum().nlargest(3).index
top_states_monthly_sales = policies_by_state_month[policies_by_state_month['state'].isin(top_3_states)]

# Check intermediate outputs
print("Top 3 states:", top_3_states)
print(top_states_monthly_sales.head())

# Pivot the table
top_states_pivot = top_states_monthly_sales.pivot(index='state', columns='month', values='policy_count').fillna(0).astype(int)

print("\nPolicies sold by month for top 3 states:")
print(top_states_pivot)


Top 3 states: Index([], dtype='object', name='state')
Empty DataFrame
Columns: [state, month, policy_count]
Index: []

Policies sold by month for top 3 states:
Empty DataFrame
Columns: []
Index: []
